In [ ]:
pip install opencv-python

In [ ]:
import numpy as np

In [ ]:
import cv2
from IPython.display import Image, display

In [ ]:
# Load the images

frame_1 = cv2.imread("Frame1.png")
frame_2 = cv2.imread("Frame2.png")

In [ ]:
# Convert the images to grayscale as SIFT works on grayscale images
frame_1_gray = cv2.cvtColor(frame_1, cv2.COLOR_BGR2GRAY)
frame_2_gray = cv2.cvtColor(frame_2, cv2.COLOR_BGR2GRAY)

In [ ]:
"""
for i in range(3,13,2):
  frame_gray_test_smoothed = cv2.GaussianBlur(frame_2_gray, (i * 5, i * 5), 0)
  sift_test = cv2.SIFT_create()
  keypoints, _ = sift_test.detectAndCompute(frame_gray_test_smoothed, None)
  test =cv2.drawKeypoints(frame_2, keypoints, outImage=None, color=(0, 255, 255),flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
  cv2.imwrite(f"test_{i}.jpg", test)
  """

'\nfor i in range(3,13,2):\n  frame_gray_test_smoothed = cv2.GaussianBlur(frame_2_gray, (i * 5, i * 5), 0)\n  sift_test = cv2.SIFT_create()\n  keypoints, _ = sift_test.detectAndCompute(frame_gray_test_smoothed, None)\n  test =cv2.drawKeypoints(frame_2, keypoints, outImage=None, color=(0, 255, 255),flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)\n  cv2.imwrite(f"test_{i}.jpg", test)\n  '

In [ ]:
# Smooth images
frame_1_gray_smoothed = cv2.GaussianBlur(frame_1_gray, (35,35), 0)
frame_2_gray_smoothed = cv2.GaussianBlur(frame_2_gray, (35,35), 0)

In [ ]:
# Check the blurred images
cv2.imwrite("frame_1_smoothed.jpg", frame_1_gray_smoothed)
cv2.imwrite("frame_2_smoothed.jpg", frame_2_gray_smoothed)
cv2.imwrite("frame_1_presmooethed.jpg", frame_1_gray)

True

In [ ]:
# Initialize the SIFT detectors

sift_1 = cv2.SIFT_create()
# sift_1.setGridSpacing(10)
sift_2 = cv2.SIFT_create()

In [ ]:
keypoints_1, descriptors_1 = sift_1.detectAndCompute(frame_1_gray_smoothed, None)
keypoints_2, descriptors_2 = sift_2.detectAndCompute(frame_2_gray_smoothed, None)
image_1_with_keypoints =cv2.drawKeypoints(frame_1, keypoints_1, outImage=None, color=(0, 255, 255),flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
image_2_with_keypoints =cv2.drawKeypoints(frame_2, keypoints_2, outImage=None, color=(0, 255, 255), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
cv2.imwrite("image_1_with_keypoints.jpg", image_1_with_keypoints)
cv2.imwrite("image_2_with_keypoints.jpg", image_2_with_keypoints)

True

In [ ]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors_1, descriptors_2, k=2)
good = []

for m, n in matches:
  if m.distance < 0.5 * n.distance:
    good.append([m])

matching_image = cv2.drawMatchesKnn(frame_1, keypoints_1, frame_2, keypoints_2, good, None, flags=2)
cv2.imwrite("matching_image.png", matching_image)


True

In [ ]:
matches = [i[0] for i in good]
matches_sorted = sorted(matches, key = lambda x : x.distance)
points_1 = np.zeros((len(matches), 2), dtype=np.float32)
points_2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches_sorted):
  points_1[i,:] = keypoints_1[match.queryIdx].pt
  points_2[i,:] = keypoints_2[match.trainIdx].pt


In [ ]:
F, mask = cv2.findFundamentalMat(points_1, points_2, cv2.FM_8POINT)
F


array([[-3.29496585e-07,  4.39018701e-05, -9.27508001e-03],
       [-4.19662391e-05,  1.75418023e-06, -1.85197823e-03],
       [ 8.98740768e-03, -3.23526490e-03,  1.00000000e+00]])

In [ ]:
x_3d = np.array([points_1[1][0], points_1[1][1], 1])
x_3d_ = np.array([points_2[1][0], points_2[1][1], 1])
x_3d_ @ F @ x_3d

0.07112416439062486

In [ ]:
intrinsic_matrix_1 = np.array([[1.600e+03, 0.000e+00, 9.595e+02],
[0.000e+00, 1.600e+03, 5.395e+02],
[0.000e+00, 0.000e+00, 1.000e+00]])

extrinsic_matrix_1 = np.array([[-6.32422984e-01, -7.74574101e-01, 8.72639567e-03, -2.36433081e+00],
[-5.00836670e-01, 4.00276423e-01, -7.67425179e-01, -1.74806440e+00],
[ 5.90934694e-01, -4.89707828e-01, -6.41079128e-01, 2.59576015e+01],
[ 0., 0., 0., 1.]])

intrinsic_matrix_2 = np.array([[1.49333333e+03, 0.00000000e+00, 9.78700000e+02],
[0.00000000e+00, 1.49333333e+03, 5.20300000e+02],
[0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]
)

extrinsic_matrix_2 = np.array([[-0.5845883, -0.81050003, -0.03669427, -2.39520617],
[-0.5041514, 0.39832053, -0.76627171, -1.79913743],
[ 0.6356793, -0.42945388, -0.64146805, 26.26804151],
[ 0., 0., 0., 1.]])

R_vec = np.array([[ 0.9971792 , -0.00348069, 0.07497863],
[ 0.00362138, 0.99999203, -0.00174044],
[-0.07497205, 0.00200697, 0.99718366]]
)

T_vec = np.array( [-1.98989457, 0.00265269, 0.20979508])
R_vec


array([[ 0.9971792 , -0.00348069,  0.07497863],
       [ 0.00362138,  0.99999203, -0.00174044],
       [-0.07497205,  0.00200697,  0.99718366]])

In [ ]:
skew = intrinsic_matrix_2 @ T_vec
skew = np.array([[0, -skew[-1], skew[1]], [skew[-1], 0, -skew[0]], [-skew[1], skew[0], 0]])
intrinsic_matrix_1_inv = np.linalg.inv(intrinsic_matrix_1)
F_other = skew @ intrinsic_matrix_2 @ R_vec @ intrinsic_matrix_1_inv
acc_matrix = F_other / F_other[2][2]
acc_matrix


array([[ 7.64315161e-09,  1.67264739e-06, -9.48129529e-04],
       [-4.78516067e-07, -2.60194129e-08, -2.50407946e-02],
       [ 3.58785894e-04,  2.20686928e-02,  1.00000000e+00]])

In [ ]:
x_3d = np.array([points_1[7][0], points_1[7][1], 1])
x_3d_ = np.array([points_2[7][0], points_2[7][1], 1])
x_3d_ @ acc_matrix @ x_3d

-0.024798437432918874

In [ ]:
# here I will try do 3 (d)

threshold = 0.06

new_points_1 = []
new_points_2 = []
new_matches = []
print(len(points_1))
print()

for i in range(len(points_1)):
  x_3d = np.array([points_1[i][0], points_1[i][1], 1])
  x_3d_ = np.array([points_2[i][0], points_2[i][1], 1])
  result = x_3d_ @ acc_matrix @ x_3d
  if abs(result) < threshold:
    new_points_1.append(points_1[i])
    new_points_2.append(points_2[i])
    new_matches.append(matches_sorted[i])
print(len(new_points_1))

new_matches = [[i] for i in new_matches]
matching_image_corrected = cv2.drawMatchesKnn(frame_1, keypoints_1, frame_2, keypoints_2, new_matches, None, flags=2)
cv2.imwrite("matching_image_corrected.png", matching_image_corrected)




57

28


True

In [ ]:
threshold = 0.08

new_points_1 = []
new_points_2 = []
new_matches = []
print(len(points_1))
print()

for i in range(len(points_1)):
  x_3d = np.array([points_1[i][0], points_1[i][1], 1])
  x_3d_ = np.array([points_2[i][0], points_2[i][1], 1])
  result = x_3d_ @ F @ x_3d
  if abs(result) < threshold:
    new_points_1.append(points_1[i])
    new_points_2.append(points_2[i])
    new_matches.append(matches_sorted[i])
print(len(new_points_1))

57

8


In [ ]:
matches = [i[0] for i in new_matches]
matches_sorted = sorted(matches, key = lambda x : x.distance)
points_1 = np.zeros((len(matches), 2), dtype=np.float32)
points_2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches_sorted):
  points_1[i,:] = keypoints_1[match.queryIdx].pt
  points_2[i,:] = keypoints_2[match.trainIdx].pt

TypeError: ignored

In [ ]:
H, _ = cv2.findHomography(points_1, points_2, cv2.RANSAC)
rectified_image = cv2.warpPerspective(frame_1, H, (frame_2.shape[1], frame_2.shape[0]))
combined_image = np.concatenate((rectified_image, frame_2), axis=1)
cv2.imwrite("rectified.png", rectified_image)
cv2.imwrite("combined.png", combined_image)
points_1 = np.array(points_1, dtype=np.float32).reshape(-1, 1, 2)
points_rectified = cv2.perspectiveTransform(points_1, H)
points_2 = np.array(points_2, dtype=np.float32).reshape(-1, 1, 2)
new = cv2.circle(rectified_image, tuple([int(i) for i in points_rectified[0][0]]), 5, (0,255,0),-1)
new_2 = cv2.circle(frame_2, tuple([int(i) for i in points_2[0][0]]), 5, (0,255,0),-1)
cv2.imwrite("new.png", new)
cv2.imwrite("new_2.png", new_2)
la = cv2.line(frame_2, tuple([int(i) for i in points_2[0][0]]), rectified_image, tuple([int(i) for i in points_rectified[0][0]]), (0, 255, 0), 2)
cv2.imwrite("la.png", la)

In [ ]:
#disparity map

stereo = cv2.StereoBM_create(numDisparities = 64)
disparity = stereo.compute(frame_1_gray, frame_2_gray)
cv2.imwrite("disparity.png", disparity)

In [ ]:
edges = cv2.Canny(frame_1_gray, 50, 300)
cv2.imwrite("edges.png", edges)
frame_1.shape

In [ ]:
_, thresh = cv2.threshold(frame_2_gray, 50, 150, cv2.THRESH_BINARY)
cv2.imwrite("thresh.png",thresh)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_img = cv2.drawContours(frame_2, contours, -1, (0,100,0), 2)
cv2.imwrite("contours.png",contours_img)

In [ ]:
import cv2
import numpy as np

# Load the stereo images (left and right frames)


# Define the calibration matrices for both cameras (K1, K2), and their distortion coefficients (D1, D2).
K1 = intrinsic_matrix_1
K2 = intrinsic_matrix_2
D1 = np.array([0, 0, 0, 0])
D2 = np.array([0, 0, 0, 0])
# Also, specify the relativeblob:https://colab.research.google.com/2b69f238-7237-48db-8d10-e7ddc38a813f transformation between the cameras (R, T).
R = R_vec
T = T_vec
# Ensure that these matrices have been correctly obtained during camera calibration.

# Rectify the images
flags = 0
alpha = 0  # Set alpha to 0 for no extra pixels in the rectified images (only valid pixels)

# Rectify the images using cv2.stereoRectify and cv2.initUndistortRectifyMap
image_width =  1920
image_height = 1080
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(K1, D1, K2, D2, (image_width, image_height), R, T, flags, alpha, (0, 0))
map1x, map1y = cv2.initUndistortRectifyMap(K1, D1, R1, P1, (image_width, image_height), cv2.CV_32F)
map2x, map2y = cv2.initUndistortRectifyMap(K2, D2, R2, P2, (image_width, image_height), cv2.CV_32F)

# Remap the images to rectify them
rectified_left = cv2.remap(frame_1_gray, map1x, map1y, interpolation=cv2.INTER_LINEAR)
rectified_right = cv2.remap(frame_2_gray, map2x, map2y, interpolation=cv2.INTER_LINEAR)
cv2.imwrite("rectified_left.png",rectified_left)
cv2.imwrite("rectified_right.png",rectified_right)

#check_point = cv2.circle(rectified_right, (375, 630), color=(0,0,255), radius=10)
check_point = cv2.circle(rectified_right, (374, 630), color=(0,0,255), radius=5)
cv2.imwrite("check_point.png",check_point)

#check_point_left = cv2.circle(rectified_left, (525, 630), color=(0,0,255), radius=10)
check_point_left = cv2.circle(rectified_left, (522, 630), color=(0,0,255), radius=5)
cv2.imwrite("check_point_left.png",check_point_left)

#check_point_2 = cv2.circle(rectified_right, (610, 525), color=(0,0,255), radius=10)
check_point_2 = cv2.circle(rectified_right, (614, 526), color=(0,0,255), radius=5)
cv2.imwrite("check_point_2.png",check_point_2)

#check_point_left_2 = cv2.circle(rectified_left, (755, 525), color=(0,0,255), radius=10)
check_point_left_2 = cv2.circle(rectified_left, (752, 526), color=(0,0,255), radius=5)
cv2.imwrite("check_point_left_2.png",check_point_left_2)

#check_point_3 = cv2.circle(rectified_right, (1025, 970), color=(0,0,255), radius=10)
check_point_3 = cv2.circle(rectified_right, (1027, 975), color=(0,0,255), radius=5)
cv2.imwrite("check_point_3.png",check_point_3)

#check_point_left_3 = cv2.circle(rectified_left, (1210, 970), color=(0,0,255), radius=10)
check_point_left_3 = cv2.circle(rectified_left, (1210, 975), color=(0,0,255), radius=5)
cv2.imwrite("check_point_left_3.png",check_point_left_3)

#check_point_4 = cv2.circle(rectified_right, (1632, 343), color=(0,0,255), radius=10)
check_point_4 = cv2.circle(rectified_right, (1651, 263), color=(0,0,255), radius=5)
cv2.imwrite("check_point_4.png",check_point_4)

#check_point_left_4 = cv2.circle(rectified_left, (1743, 343), color=(0,0,255), radius=10)
check_point_left_4 = cv2.circle(rectified_left, (1763, 263), color=(0,0,255), radius=5)

cv2.imwrite("check_point_left_4.png",check_point_left_4)

#check_point_5 = cv2.circle(rectified_right, (973, 715), color=(0,0,255), radius=10)
check_point_5 = cv2.circle(rectified_right, (969, 625), color=(0,0,255), radius=5)
cv2.imwrite("check_point_5.png",check_point_5)

#check_point_left_5 = cv2.circle(rectified_left, (1120, 715), color=(0,0,255), radius=10)
check_point_left_5 = cv2.circle(rectified_left, (1124, 625), color=(0,0,255), radius=5)
cv2.imwrite("check_point_left_5.png",check_point_left_5)

combined_image = np.concatenate((rectified_left, rectified_right), axis=1)
#check_combined = cv2.line(combined_image, (0, 700), (4000, 700), color=(0,0,255))
#cv2.imwrite("check_combined.png",check_combined)
cv2.imwrite("combined_image.png",combined_image)

stereo = cv2.StereoBM_create(numDisparities=96, blockSize=15)
disparity = stereo.compute(rectified_left, rectified_right)
cv2.imwrite("disparity.png",disparity)
focal_length_av = (intrinsic_matrix_1[0,0] + intrinsic_matrix_2[0,0]) / 2
disparity = abs(disparity)

print(Q)
points_3d = cv2.reprojectImageTo3D(disparity, Q)

# Calculate the disparity map using StereoSGBM or another stereo matching algorithm
# You need to specify the disparity range and matching parameters appropriately
"""
window_size = 5
min_disp = 0
num_disp = 128

stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=window_size,
    P1=8 * 3 * window_size ** 2,
    P2=32 * 3 * window_size ** 2,
)

disparity = stereo.compute(rectified_left, rectified_right)
baseline = np.linalg.norm(T)

# Convert the disparity map to depth using the calibration parameters
focal_length = P1[0, 0]  # Focal length from the rectified camera matrix P1

depth_map = (focal_length * baseline) / (disparity + 1e-6)

cv2.imwrite("depth_map.png",depth_map)
# At this point, you can continue with the segmentation and pool area estimation steps as previously described.
"""

print(P1)
print(P2)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -1.11699886e+03]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -5.29627289e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.54666666e+03]
 [ 0.00000000e+00  0.00000000e+00  4.99768822e-01 -0.00000000e+00]]
[[1.54666666e+03 0.00000000e+00 1.11699886e+03 0.00000000e+00]
 [0.00000000e+00 1.54666666e+03 5.29627289e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
[[ 1.54666666e+03  0.00000000e+00  1.11699886e+03 -3.09476421e+03]
 [ 0.00000000e+00  1.54666666e+03  5.29627289e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]


In [ ]:
"""
point1 = np.array([7.8932, -1.3383, -20.6213])
point2 = np.array([4.9929, 0.06382, -21.3324])
point3 = np.array([-1.00542, -4.76078, -16.72])

# Calculate the vectors of the sides of the triangle
vector1 = point1 - point2
vector2 = point3 - point2

# Calculate the cross product of the vectors
cross_product = np.cross(vector1, vector2)

# Calculate the magnitude of the cross product
magnitude = np.linalg.norm(cross_product)


area = magnitude

print('Area:', area)
"""

Area: 28.677296246571846


In [ ]:
point1 = np.array([8.0404, -1.3567, -20.891])
point2 = np.array([5.288, 0.0525, -22.414])
point3 = np.array([-1.0164, -4.867, -16.902])

# Calculate the vectors of the sides of the triangle
vector1 = point1 - point2
vector2 = point3 - point2

# Calculate the cross product of the vectors
cross_product = np.cross(vector1, vector2)

# Calculate the magnitude of the cross product
magnitude = np.linalg.norm(cross_product)


area = magnitude

print('Area:', area)

Area: 33.41603105832077


In [ ]:
point1 = np.array([-0.0408, -2.5306, -21.04217])
point2 = np.array([-11.279, 3.3625, -27.86])

# Calculate the Euclidean distance
distance = np.linalg.norm(point1 - point2)

print('Distance:', distance)

Distance: 14.405157852620011


In [ ]:
point1 = np.array([-11.5357, 4.7611, -27.617])
point2 = np.array([-0.09034, -1.2306, -19.9561])

# Calculate the Euclidean distance
distance = np.linalg.norm(point1 - point2)

print('Distance:', distance)

Distance: 15.01952473381232
